# LWE and RLWE encryption
## LWE encryption

In FHEW-like HE, we use both LWE and RLWE encryption.
LWE is for a ciphertext and RLWE is for the core part of bootstrapping so called *blind rotation*.

We define LWE encryption as follows
$$\texttt{LWE}_{\vec{s}, n, q}(m) = (\beta, \vec{\alpha}) \in \mathbb{Z}_q^{n+1},$$
where $\beta = m + e - \left< \vec{\alpha}, \vec{s} \right> \in \mathbb{Z}_q$, and $\vec{s} \leftarrow \chi_{key}$ is a secret key and $\vec{e} \leftarrow \chi_{err}$ is a added noise for security.
$\vec{\alpha}$ is unifromly sampled in $\mathbb{Z}_q^n$.

Here, we choose $\chi_{key}$ as binary distribution and $\chi_{err}$ as a Gaussian distribution with standard deviation $3.2$.

Let's make the encryption method. We use pytorch for easy and fast implementation.

In [1]:
import torch
import numpy as np


We use parameter sets $(n, q, \sigma) = (512, 2048, 3.2)$.

In [2]:
stddev = 3.2
n = 512
q = 2048

Following are generator of key and error.

NOTE: Those generators are not secure. You should **NOT** use them in practice.

In [3]:
def keygen(dim):
    return torch.randint(2, size = (dim,))
def errgen(stddev):
    e = torch.round(stddev*torch.randn(1))
    e = e.squeeze()
    return e.to(torch.int)
def uniform(dim, modulus):
    return torch.randint(modulus, size = (dim,))

We first generate the secret key $\vec{s}$.

In [4]:
s = keygen(n)
s

tensor([1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1,
        1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
        1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1,
        0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
        0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1,
        1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1,
        1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1,
        1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1,
        0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0,
        1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1,
        1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1,
        0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1,

To encrypt, we need random part $\vec{\alpha}$.

In [5]:
alpha = uniform(n, q)
alpha

tensor([ 213, 1767, 1877,  371,  535, 1617,  481, 1841, 1424, 1921, 1704,  800,
        1873,   35,  217,  687,  872,  755, 1019, 1272,  444,  775,   80, 1844,
        1597,  652,  728, 1909, 1831,  798,  636,  806, 1776,  594, 1292, 1783,
        1084,  410, 1416, 2038,  235, 1397, 1629,  873, 1923, 1968,  726,  353,
         738,  520, 1769,   33,  292, 1734, 1438, 1150, 1908, 1113,  559, 1860,
         557, 1625, 1118, 1276, 1112, 1366,  547, 1497,  326, 1450, 1693, 1830,
        1721, 2012,  856, 1210,  306,  802,  160,  741, 1966, 1618, 1360,   75,
        1708, 1360, 1561,  900,  693, 1770,   49,  920,  631, 1865,  273, 1230,
        1111, 1516,  857, 1562, 1663,  182, 1356,   47,   49, 1253, 1196, 1963,
         513,   51, 1298,  415,  104, 1967,   75,  920, 1841,  159,  950,   79,
        1579, 1875, 1992,  470, 1858, 1880, 1223,   23, 1844,  619,   45, 1749,
         100,  401,  629, 2036,  674,  437, 1726, 2004,  217,  254, 1491, 1836,
         757,  158,  602,  621,  267,  6

We calaulate $\beta = m + e - \left< \vec{\alpha}, \vec{s} \right>$ for encryption.

Let the message we are encrypting is a binary value e,g, $m = 1$ here.

In [6]:
m = 1

beta = m - torch.dot(alpha, s)
e = errgen(stddev)
beta += e

beta %= q

beta

tensor(1808)

Let's decrypt the ciphertext above.

As $\beta = m + e - \left< \vec{\alpha}, \vec{s} \right>$, we can find $m + e = \beta + \left< \vec{\alpha}, \vec{s} \right>$.

In [7]:
m_decrypted = beta + torch.dot(alpha, s)
m_decrypted %= q
m_decrypted

tensor(0)

If you are very lucky, you might get the decrypted value.
```
>>> m_decrypted
tensor(1)
```
But, if you run the code once, you will get other value.
Note here that we get $m+e$ by decryption not exactly the value $m$.

To make our message safe from the error, we can multiply certain *scaling factor* to our message.

Here, let's multiply $q/4$, and encrypt/decrypt again.


In [8]:
m = 1
# multiply scaling factor q/4 
m *= q//4

beta = m - torch.dot(alpha, s)
e = errgen(stddev)
beta += e
beta %= q

m_decrypted = beta + torch.dot(alpha, s)
m_decrypted %= q

m_decrypted

tensor(511)

We got a value near $m \cdot q/4 = 512$.
Division by $q/4$ and rounding will give us original message.

In [9]:
# rescale the message
m_decrypted = m_decrypted.to(torch.float)
m_decrypted /= q/4.
m_decrypted = torch.round(m_decrypted)
m_decrypted.to(torch.int)


tensor(1, dtype=torch.int32)

Decryption is successful!

### LWE encryption function

By *LWE assumption* $\beta$ should look like a random value.

The LWE ciphertext is a pair $(\beta, \vec{\alpha})$.

We define the encryptor as follows.

In [10]:
def encryptLWE(message, dim, modulus, key):
    alpha = uniform(n, modulus)

    beta = message - torch.dot(alpha, s)
    e = errgen(stddev)
    beta += e
    beta %= modulus

    return (beta, alpha)


ct = encryptLWE(1, n, q, s)
ct    


(tensor(478),
 tensor([ 144, 1380,  710,  783,  194,  497,  997,  215,   13, 1299,  764, 1392,
         1718, 1799,  502, 1746, 1747,  134,  751,  752,  634, 1738, 1121,  680,
          887, 1544,   85,  224,  224,  694, 1211, 1571, 1703, 1191,  835, 1394,
         1447,  909, 2027, 1049, 1164,  276,  700, 1668, 1330, 1691,  968, 1359,
         1805,   79, 1940,  444,  611,   90,  344, 1451, 1279,  799, 1161, 1942,
           98, 1660,  620, 2037, 1843,  563,  821, 1139, 1439, 1978, 1723,  157,
         1773, 1718,  747,  716,  319, 1616, 1321,  864, 1705, 1670,  896, 1423,
          663,  933, 1399, 1944,  328, 1512,  870,  129,  640,  172,  475,  177,
         1600, 1623, 1887, 1785,  825, 1975, 1431,  813, 1276,  394, 1794,  635,
          884,  415, 1684, 1709, 1042, 1549,  254,  238,  939,   20,  684,  273,
          114, 1282,  851,    7, 1934, 1014,  213,  996, 1263, 1748, 1232, 1872,
          935,  234,  398, 1427,  276,   31,  188, 2040,  790, 1528,  305,  510,
         1815,